<a href="https://colab.research.google.com/github/newmantic/Black_Litterman/blob/main/Black_Litterman_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize

def calculate_implied_returns(cov_matrix, market_weights, risk_aversion):
    """
    Calculate the implied equilibrium excess returns.

    :param cov_matrix: Covariance matrix of asset returns
    :param market_weights: Market capitalization weights vector
    :param risk_aversion: Risk aversion coefficient (lambda)
    :return: Implied equilibrium returns vector (Pi)
    """
    return risk_aversion * np.dot(cov_matrix, market_weights)

def black_litterman_expected_returns(implied_returns, cov_matrix, tau, P, Q, omega):
    """
    Calculate the Black-Litterman expected returns incorporating investor views.

    :param implied_returns: Implied equilibrium returns vector (Pi)
    :param cov_matrix: Covariance matrix of asset returns (Sigma)
    :param tau: Scalar reflecting uncertainty in prior estimate of mean returns
    :param P: Matrix representing investor views (linking views to assets)
    :param Q: Vector of expected returns from investor views
    :param omega: Diagonal covariance matrix of error terms in investor views
    :return: New expected returns vector (mu)
    """
    M_inverse = np.linalg.inv(np.dot(np.dot(P, tau * cov_matrix), P.T) + omega)
    adj_returns = np.dot(tau * cov_matrix, np.dot(P.T, np.dot(M_inverse, Q - np.dot(P, implied_returns))))
    return implied_returns + adj_returns

def optimize_portfolio(expected_returns, cov_matrix, risk_aversion):
    """
    Optimize the portfolio weights using mean-variance optimization.

    :param expected_returns: New expected returns vector (mu)
    :param cov_matrix: Covariance matrix of asset returns (Sigma)
    :param risk_aversion: Risk aversion coefficient (lambda)
    :return: Optimal portfolio weights (w*)
    """
    return np.dot(np.linalg.inv(cov_matrix), expected_returns) / risk_aversion

# Testable Example

# Covariance matrix of 3 assets
cov_matrix = np.array([[0.005, -0.010, 0.004],
                       [-0.010, 0.040, -0.002],
                       [0.004, -0.002, 0.023]])

# Market capitalization weights (as percentages)
market_weights = np.array([0.4, 0.3, 0.3])

# Risk aversion coefficient (lambda)
risk_aversion = 3.0

# Calculate implied equilibrium returns (Pi)
implied_returns = calculate_implied_returns(cov_matrix, market_weights, risk_aversion)
print(f"Implied Equilibrium Returns (Pi): {implied_returns}")

# Investor views: P matrix and Q vector
P = np.array([[1, 0, -1],  # View: Asset 1 will outperform Asset 3
              [0, 1, -1]])  # View: Asset 2 will outperform Asset 3
Q = np.array([0.05, 0.03])  # Investor expects a 5% return difference for View 1, and 3% for View 2

# Omega: Uncertainty in investor views
omega = np.diag([0.0001, 0.0001])

# Scalar tau
tau = 0.025

# Calculate Black-Litterman expected returns
bl_expected_returns = black_litterman_expected_returns(implied_returns, cov_matrix, tau, P, Q, omega)
print(f"Black-Litterman Expected Returns (mu): {bl_expected_returns}")

# Optimize the portfolio weights
optimal_weights = optimize_portfolio(bl_expected_returns, cov_matrix, risk_aversion)
print(f"Optimal Portfolio Weights: {optimal_weights}")

Implied Equilibrium Returns (Pi): [0.0006 0.0222 0.0237]
Black-Litterman Expected Returns (mu): [ 0.00408515 -0.00355426 -0.03367581]
Optimal Portfolio Weights: [ 1.41991998  0.28987155 -0.70979154]
